In [1]:
from tensorflow.keras.utils import FeatureSpace
from tensorflow.keras.models import load_model
import tensorflow as tf
import pickle
import pandas as pd
import numpy as np

import sys
import os

parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.append(parent_dir)

import helper
import json
import requests

In [2]:
reconstructed_model = load_model(parent_dir + '\\' + "model/inference_model.keras")

In [3]:
#load tfidf_vectorizer, transform the description column
fileName = parent_dir + '\\' + 'model/tfidf_vectorizer.pkl'
with open(fileName,'rb') as f:
    tfidf_object = pickle.load(f)

In [4]:
fileName = parent_dir + '\\' + 'model/price_by_hull_material.pkl'
with open(fileName,'rb') as f:
    price_by_hull_material = pickle.load(f)

average_value_hull_material = sum(price_by_hull_material.values()) / len(price_by_hull_material)

fileName = parent_dir + '\\' + 'model/price_by_fuel_type.pkl'
with open(fileName,'rb') as f:
    price_by_fuel_type = pickle.load(f)

average_value_fuel_type = sum(price_by_fuel_type.values()) / len(price_by_fuel_type)

fileName = parent_dir + '\\' + 'model/price_by_category.pkl'
with open(fileName,'rb') as f:
    price_by_category = pickle.load(f)

average_value_category = sum(price_by_category.values()) / len(price_by_category)

In [5]:
input_dict = {
    'year': 2010.0,
    'model': 'Sailing Yacht',
    'category':'Slup Boote',
    'length': 70.,
    'fuel_type': 'Diesel',
    'hull_material': 'Steel',
    'country': 'Turkey',
    'description': 'The Futuna 70 ICHTUS has been designed for a precise purpose; a shorthanded trip around the world. Designed by famous naval architect Marc Lombard, she proved to be a quick and (very) beautiful sailing yacht. She is also comfortable, fast and easy to handle despite her length, with fresh water and energy self-sufficiency.Under her new ownership since the end of 2016, ICHTUS undertook a major refit with a complete refurbishment & upgrade in 2017',
}

In [6]:
sample, pred_dict = helper.build_pred_dict(input_dict, tfidf_object, price_by_hull_material, price_by_fuel_type, price_by_category)

In [7]:
sample

{'year': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([2010.], dtype=float32)>,
 'model': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'sailing_yacht'], dtype=object)>,
 'category': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'slup_boote'], dtype=object)>,
 'length': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([70.], dtype=float32)>,
 'fuel_type': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'diesel'], dtype=object)>,
 'hull_material': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'steel'], dtype=object)>,
 'country': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'turkey'], dtype=object)>,
 'noun_counts': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([23.], dtype=float32)>,
 'verb_counts': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([1.], dtype=float32)>,
 'adjective_counts': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([12.], dtype=float32)>,
 'bilge_pump': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.], dtype=f

In [8]:
reconstructed_model.predict(sample)

1/1 [==============================] - 1s 511ms/step


array([[1995156.5]], dtype=float32)

In [9]:
url = 'http://localhost:8000/predict'

In [10]:
response = requests.post(
    url,
    json=input_dict,  # Serialize the input_dict as JSON
    headers={"Content-Type": "application/json"}  # Set the appropriate Content-Type header
)

In [11]:
if response.status_code == 200:
    print("Request was successful")
else:
    print("Request failed with status code:", response.status_code)

Request was successful


In [12]:
response.json()

{'predicted_price': '1995156.5'}